<a href="https://colab.research.google.com/github/ahyds/Machine-Learning-Project/blob/master/Cloud%20ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

!pip install pyspark

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.postgresql:postgresql:42.1.1 pyspark-shell'

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
     |████████████████████████████████| 215.7MB 59kB/s 
     |████████████████████████████████| 204kB 45.7MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=3d40181cec764471bb7b2b3e85587527dabfb7f3170756995781819b06ccb36d
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


# Read in CSV from S3

In [0]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url_1990 = "https://hyd123.s3.us-east-2.amazonaws.com/updated_data/1990data_cleaned.csv"
url_2000 = "https://hyd123.s3.us-east-2.amazonaws.com/updated_data/2000data_cleaned.csv"
url_2010 = "https://hyd123.s3.us-east-2.amazonaws.com/updated_data/2010data_cleaned.csv"
url_2019 = "https://hyd123.s3.us-east-2.amazonaws.com/updated_data/2019data_cleaned.csv"

In [0]:
spark.sparkContext.addFile(url_1990)
df_1990 = spark.read.option('header', 'true').csv(SparkFiles.get("1990data_cleaned.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

In [0]:
spark.sparkContext.addFile(url_2000)
df_2000 = spark.read.option('header', 'true').csv(SparkFiles.get("2000data_cleaned.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

In [0]:
spark.sparkContext.addFile(url_2010)
df_2010 = spark.read.option('header', 'true').csv(SparkFiles.get("2010data_cleaned.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

In [0]:
spark.sparkContext.addFile(url_2019)
df_2019 = spark.read.option('header', 'true').csv(SparkFiles.get("2019data_cleaned.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

# Check Data Schema

In [7]:
df_1990.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- analysis_url: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- energy: double (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- key: integer (nullable = true)
 |-- liveness: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: integer (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: integer (nullable = true)
 |-- track_href: string (nullable = true)
 |-- type: string (nullable = true)
 |-- uri: string (nullable = true)
 |-- valence: double (nullable = true)



In [8]:
df_1990.show(10)

+---+------------+--------------------+------------+-----------+------+--------------------+----------------+---+--------+--------+----+-----------+-------+--------------+--------------------+--------------+--------------------+-------+
|_c0|acousticness|        analysis_url|danceability|duration_ms|energy|                  id|instrumentalness|key|liveness|loudness|mode|speechiness|  tempo|time_signature|          track_href|          type|                 uri|valence|
+---+------------+--------------------+------------+-----------+------+--------------------+----------------+---+--------+--------+----+-----------+-------+--------------+--------------------+--------------+--------------------+-------+
|  0|       0.736|https://api.spoti...|        0.78|     322133| 0.565|1NCuYqMc8hKMb4cpN...|         0.00269|  3|   0.063|   -7.32|   0|     0.0306|101.967|             4|https://api.spoti...|audio_features|spotify:track:1NC...|  0.365|
|  1|      0.0561|https://api.spoti...|       0.631|

# Write DataFrame to RDS

In [0]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://monpostgresqldb.c1rttjrklnzf.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root",
          "password": "password",
          "driver":"org.postgresql.Driver"}

In [0]:
# Write DataFrames to table
df_1990.write.jdbc(url=jdbc_url, table='df_1990', mode=mode, properties=config)

In [0]:
df_2000.write.jdbc(url=jdbc_url, table='df_2000', mode=mode, properties=config)

In [0]:
df_2010.write.jdbc(url=jdbc_url, table='df_2010', mode=mode, properties=config)

In [0]:
df_2019.write.jdbc(url=jdbc_url, table='df_2019', mode=mode, properties=config)